In [3]:
import pandas as pd

df = pd.read_csv('data/results-gpt4o-mini.csv')

In [4]:
df = df.iloc[:300]

In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
answer_llm = df.iloc[0].answer_llm

In [8]:
v_q = embedding_model.encode(answer_llm)

In [10]:
v_q[0]

-0.42244676

In [11]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [12]:
evaluations = []
for i in range(len(df)):
    v_llm = embedding_model.encode(df.iloc[i].answer_llm)
    v_orig = embedding_model.encode(df.iloc[i].answer_orig)

    evaluations.append(v_llm.dot(v_orig))

In [14]:
import numpy as np
np.percentile(evaluations, 75)

31.674304008483887

In [15]:
def norm(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [16]:
evaluations_norm = []
for i in range(len(df)):
    v_llm = embedding_model.encode(df.iloc[i].answer_llm)
    v_llm = norm(v_llm)
    v_orig = embedding_model.encode(df.iloc[i].answer_orig)
    v_orig = norm(v_orig)

    evaluations_norm.append(v_llm.dot(v_orig))

In [17]:
np.percentile(evaluations_norm, 75)

0.8362347185611725

In [20]:
from rouge import Rouge
rouge_scorer = Rouge()

In [21]:
def compute_rouge(row):
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    return pd.Series({
        'rouge1_precision': scores['rouge-1']['p'],
        'rouge1_recall': scores['rouge-1']['r'],
        'rouge1_fmeasure': scores['rouge-1']['f'],
        'rouge2_precision': scores['rouge-2']['p'],
        'rouge2_recall': scores['rouge-2']['r'],
        'rouge2_fmeasure': scores['rouge-2']['f'],
        'rougeL_precision': scores['rouge-l']['p'],
        'rougeL_recall': scores['rouge-l']['r'],
        'rougeL_fmeasure': scores['rouge-l']['f']
    })

In [24]:
rouge_scores = df.apply(compute_rouge, axis=1)

result_df = pd.concat([df, rouge_scores], axis=1)

In [35]:
result_df.iloc[10]['rouge1_fmeasure']

0.45454544954545456

In [39]:
result_df.iloc[10][['rouge1_fmeasure', 'rouge2_fmeasure', 'rougeL_fmeasure']].mean()

0.35490034990035496

In [41]:
df_rouge_2 = result_df[['rouge2_precision', 'rouge2_recall', 'rouge2_fmeasure']]

In [43]:
df_rouge_2.values.mean()

0.22140135503422884